In [40]:
##importing libraries
import pandas as pd
import nltk
import numpy as np
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report,accuracy_score,confusion_matrix
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

,0,1
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will ü b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [41]:
###reading the data
data = pd.read_csv('/Users/drdre/Downloads/Spam classification/smsspamcollection/SMSSpamCollection',sep='\t',header=None)
data.columns = ['label',"text"]
data.head()

,label,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [42]:
####Data Cleaning
from nltk.stem.porter import PorterStemmer
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /Users/drdre/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [43]:
##Stopwords
stopwords.words('English')  

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [44]:
stop_words = ['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each',
 'few',
 'more',
 'most',
 'other',
 'some',
 'such',
 'no',
 'nor',
 'not',
 'only',
 'own',
 'same',
 'so',
 'than',
 'too',
 'very',
 's',
 't',
 'can',
 'will',
 'just',
 'don',
 "don't",
 'should',
 "should've",
 'now',
 'd',
 'll',
 'm',
 'o',
 're',
 've',
 'y',
 'ma',
 'mightn',
 "mightn't",
 'mustn',
 "mustn't",
 'needn',
 "needn't",
 'shan',
 "shan't",
 'shouldn',
 "shouldn't",
 'wasn',
 "wasn't",
 'weren',
 "weren't",
 'won',
 "won't",
 'wouldn',
 "wouldn't"]


In [45]:
ps = PorterStemmer() ##obj for stemming
corpus = []
for i in range(0,len(data)):
    review = re.sub('[^A-Za-z0-9]',' ',data['text'][i])
    review = review.lower()
    review = review.split()
    review = [ps.stem(word) for word in review if not word in stop_words]
    review= ' '.join(review)
    corpus.append(review)
    

In [46]:
corpus

['go jurong point crazi avail bugi n great world la e buffet cine got amor wat',
 'ok lar joke wif u oni',
 'free entri 2 wkli comp win fa cup final tkt 21st may 2005 text fa 87121 receiv entri question std txt rate c appli 08452810075over18',
 'u dun say earli hor u c alreadi say',
 'nah think goe usf live around though',
 'freemsg hey darl 3 week word back like fun still tb ok xxx std chg send 1 50 rcv',
 'even brother like speak treat like aid patent',
 'per request mell mell oru minnaminungint nurungu vettam set callertun caller press 9 copi friend callertun',
 'winner valu network custom select receivea 900 prize reward claim call 09061701461 claim code kl341 valid 12 hour',
 'mobil 11 month u r entitl updat latest colour mobil camera free call mobil updat co free 08002986030',
 'gonna home soon want talk stuff anymor tonight k cri enough today',
 'six chanc win cash 100 20 000 pound txt csh11 send 87575 cost 150p day 6day 16 tsandc appli repli hl 4 info',
 'urgent 1 week free mem

In [47]:
####train test split
y= pd.get_dummies(data['label'],drop_first=True)
X_train,X_test,y_train,y_test = train_test_split(corpus,y,test_size= 0.2,random_state=0)

In [48]:
data['label'].value_counts()

ham     4825
spam     747
Name: label, dtype: int64

In [49]:
X_test

['storm msg wen u lift phne u say hello u knw wt real mean hello name girl ye u knw dat girl margaret hello girlfrnd f grahmbel invnt telphon moral one 4get name person bt girlfrnd g n g h',
 'forward 448712404000 pleas call 08712404000 immedi urgent messag wait',
 'also sorta blown coupl time recent id rather text blue look weed',
 'sir goodmorn free call',
 'come aliv better correct good look figur',
 'hous maid murder coz man murder lt gt th januari public holiday govt instituit close includ post offic understand',
 'hot n horni will live local text repli hear strt back 150p per msg netcollex ltdhelpdesk 02085076972 repli stop end',
 'sorri call later meet thing relat trade pleas call arul lt gt',
 'oh sorri pleas',
 'hey hun onbu goin 2 meet want 2go 4a meal donyt feel like cuz 2 get last bu home he sweet latelyxxx',
 'stupid auto correct phone',
 'oh k k big hitter anyway good',
 'roger probabl go rem 20',
 'ya ok vikki vl c witin lt gt min il repli u',
 'well leav class babe neve

In [50]:
###creating bag of words
cv = CountVectorizer(max_features = 2500) ###most occuring vocab words upto 2500 words max
X_train = cv.fit_transform(X_train).toarray()
X_test= cv.transform(X_test).toarray()

In [51]:
X_train.shape

(4457, 2500)

In [52]:
X_test.shape

(1115, 2500)

In [53]:
cv.vocabulary_

{'good': 878,
 'movi': 1462,
 'ok': 1580,
 'leav': 1219,
 'free': 787,
 'give': 863,
 'otherwis': 1613,
 'nalla': 1496,
 'someth': 2098,
 'mayb': 1365,
 '30': 74,
 'bit': 322,
 'hope': 997,
 'got': 885,
 'home': 991,
 'babe': 276,
 'still': 2145,
 'awak': 271,
 'kay': 1163,
 'sinc': 2057,
 'alreadi': 201,
 'workin': 2452,
 'get': 850,
 'job': 1138,
 'said': 1940,
 'matter': 1359,
 'mind': 1403,
 'say': 1956,
 'oh': 1576,
 'yeah': 2482,
 'diet': 592,
 'window': 2432,
 'sorri': 2108,
 'thing': 2239,
 'may': 1364,
 'pub': 1782,
 'later': 1205,
 'ill': 1048,
 'call': 390,
 'even': 677,
 'idea': 1039,
 'dear': 560,
 'reach': 1823,
 'room': 1915,
 'look': 1277,
 'daddi': 543,
 'want': 2390,
 'eat': 644,
 'night': 1531,
 'long': 1275,
 'chennai': 436,
 'sure': 2190,
 'pick': 1677,
 'us': 2346,
 'competit': 481,
 'da': 541,
 'car': 404,
 'park': 1636,
 'okay': 1581,
 'soon': 2105,
 'best': 308,
 'yar': 2479,
 'els': 654,
 'thk': 2242,
 'sort': 2109,
 'funni': 814,
 'yup': 2498,
 'izzit': 1112,

In [54]:
classifier = RandomForestClassifier().fit(X_train,y_train.values.ravel())

In [55]:
y_pred = classifier.predict(X_test)
y_pred

array([0, 1, 0, ..., 0, 1, 0], dtype=uint8)

In [56]:
accuracy_score(y_test,y_pred)

0.9847533632286996

In [57]:
y_train.shape

(4457, 1)

In [58]:
confusion_matrix(y_test,y_pred)

array([[955,   0],
       [ 17, 143]])

In [59]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.98      1.00      0.99       955
           1       1.00      0.89      0.94       160

    accuracy                           0.98      1115
   macro avg       0.99      0.95      0.97      1115
weighted avg       0.99      0.98      0.98      1115

